In [55]:
import gauleg as gl 
import sympy as sp 
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt
import Solver2 as sl 
from scipy.interpolate import interp1d

%load_ext autoreload

%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generating Delta Vector

In [56]:
def interpolation_for_u_h(nodal, mesh, num_points):
  """
  Given the nodal solution 'nodal' computed on the refined mesh 'mesh',
  return the solution values at observation nodes computed as a uniform grid
  over [0,1] with num_points points.

  Assumes that every observation node exactly exists in the mesh.

  Parameters:
    nodal     : array-like, the computed solution at each mesh node.
    mesh      : array-like, the coordinates of the mesh nodes.
    num_points: int, number of observation points (e.g., 8).

  Returns:
    obs_values: np.array, the solution values corresponding to each observation node.

  Raises:
    ValueError if any observation node is not found in the mesh.
  """
  nodal = sl.assemble_nodal_values(nodal)
  mesh = np.asarray(mesh)
  nodal = np.asarray(nodal)
  
  # Compute observation nodes uniformly over [0,1]
  obs_nodes = np.linspace(0.0, 1.0, num_points)
  obs_values = np.zeros(num_points)

  for i, x_obs in enumerate(obs_nodes):
    idx = np.where(mesh == x_obs)[0]
    if idx.size == 0:
      raise ValueError(f"Observation node {x_obs} not found in the mesh.")
    obs_values[i] = nodal[idx[0]]

  return obs_values


def cov_matrix(sigma, num_points):
  sigma = (sigma ** 2) * np.eye(num_points)
  return sigma



In [57]:
def add_noise(observations_at_xi, num_points, sigma):
    """
    Adds a normally distributed noise, theta
    to observations from the forward solver.

    Arguments:
    observations_at_xi : observations at predetermined xi using interplotion. 
    num_points : how big your covariance matrix is 

    Returns:
    Delta : Array of Noisy observations.
    
    """
    sigma = cov_matrix(sigma, num_points)
    noise = np.random.multivariate_normal(np.zeros(num_points), sigma)
    delta = observations_at_xi + noise 
    return delta



# Define Likelihood Function 

In [58]:
def phi(observations, predicted, sigma, num_points) :
    '''
    For a set of predetermined points xi -- obtained via np.linspace,
    this function defines the likelihood function 

    Arguments:
    observations: Generated noisy observation using beta_true -- corresponds to y in literature
    predicted: For a proposed beta_i, we compute the noisy observation using the forward solver 
    -- corresponds to g(beta_i) in literature

    Returns: 
    Likelihood function that is proportional to the prior distribution
    
    '''
    covariance_matrix = cov_matrix(sigma, num_points)
    diff = predicted - observations
    covariance_matrix_inv = (1/sigma**2) * covariance_matrix
    val = 0.5 * diff.T @ covariance_matrix_inv @ diff
    return val
# 2 cases, 1. denominator very close to 0 or both is 0. need to check if its one or both. 


In [59]:
def compute_A(phi_0, phi_i, sigma):

    val = np.exp((1/sigma**2)*(phi_0 - phi_i))
    
    return val

# MCMC algorithm 

In [60]:
def MCMC(beta_true, number_of_iter, burn_in, sigma, num_points): 
    '''
    Builds a Markov Chain 

    Key Steps:
    1. Initialise a choice of Beta, beta_0 
    2. Compute likelihood of beta_0, using delta and beta_0_predicted
    3. Initialise the loop.
        - we propose a new beta_i from x* ~ Uniform(0.15, 0.85) and r ~ Uniform(0, 0.15)
        - compute y_i and g(beta_i)
        - compute likelihood using {y_i and g(beta_i)}
        - set alpha = min{1, likelihood }
    '''
    # set seed 
    np.random.seed(42)
    # range of uniform distribution 
    x_star_range = (0.15, 0.85)
    r_range = (0, 0.13)
    chain = []
    # compute delta 
    mesh_true , c_sol_true = sl.refinement_loop(0.00001, beta_true)
    y_true = interpolation_for_u_h(c_sol_true, mesh_true, num_points)
    delta = add_noise(y_true, num_points, sigma)

    # draw first copy of beta --> beta_0

    beta_0 = np.array([
            np.random.uniform(*x_star_range),
            np.random.uniform(*r_range)
        ])
    print("Beta_0:", beta_0)
    
    # initialise current observations and likelihood 
    mesh_0, c_sol_0 = sl.refinement_loop(0.00001, beta = beta_0)
    y_0 = interpolation_for_u_h(c_sol_0, mesh_0, num_points)
    phi_0 = phi(delta, y_0, sigma, num_points)
    print("phi_0:", phi_0)
    
    iter_count = 0
    acceptance_count = 0 
    acceptance_prob_history = []



    for i in range(number_of_iter):
        beta_proposal = np.array([
            np.random.uniform(*x_star_range),
            np.random.uniform(*r_range)
        ])
        print("beta proposal:", beta_proposal)
        mesh_proposal, c_sol_proposal = sl.refinement_loop(0.00001, beta = beta_proposal)
        y_proposal = interpolation_for_u_h(c_sol_proposal, mesh_proposal, num_points)
        phi_proposal = phi(delta, y_proposal, sigma, num_points)
        print("phi proposal:", phi_proposal)
        # compute acceptance probability 
        A = compute_A(phi_0, phi_proposal, sigma)
        acceptance_prob = min(1, A)
        acceptance_prob_history.append(acceptance_prob)
        print("acceptance probablity:", acceptance_prob)
        
        # Accept or reject the proposal
        if np.random.rand() < acceptance_prob:
            beta_0 = beta_proposal # update the current state as the last accepted proposal
            y_0 = y_proposal # update the current observations to the last accepted observation
            phi_0 = phi_proposal
            acceptance_count += 1
        
        # Record the current state.
        chain.append(beta_0.copy())
        print("Chain length:", len(chain))
    
    chain = np.array(chain)
    # Compute the MCMC estimate as the mean of the samples after burn-in.
    beta_mcmc = np.mean(chain[burn_in:], axis=0)
    
    return chain, beta_mcmc, acceptance_prob_history, acceptance_count

In [63]:
number_of_iter = 1000
burn_in = 100
sigma = 0.1 
num_points = 8
beta_true = np.array([0.5, 1/6])

chain, beta_mcmc, acceptance_history, acceptance_count = MCMC(beta_true, number_of_iter, burn_in, sigma, num_points)
print("True beta:", beta_true )
print("MCMC estimated beta:", beta_mcmc)
print("Acceptance Probability History:", acceptance_history)
print("Acceptance Count:", acceptance_count)

C:\Users\Admin\AppData\Local\Temp\ipykernel_10564\2839755143.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  obs_values[i] = nodal[idx[0]]


Beta_0: [0.16440915 0.12608828]
phi_0: 0.9650144731184271
beta proposal: [0.73270985 0.02760408]
phi proposal: 0.6060549230822518
acceptance probablity: 1
Chain length: 1
beta proposal: [0.27838316 0.03955149]
phi proposal: 0.5645080107836553
acceptance probablity: 1
Chain length: 2
beta proposal: [0.45236151 0.03785979]
phi proposal: 0.39932348937242756
acceptance probablity: 1
Chain length: 3
beta proposal: [0.2476457 0.0379788]
phi proposal: 0.6215024068316654
acceptance probablity: 2.2433250981349414e-10
Chain length: 4
beta proposal: [0.46924899 0.10207287]
phi proposal: 0.12129833841320072
acceptance probablity: 1
Chain length: 5
beta proposal: [0.50996411 0.07701389]
phi proposal: 0.19646584067106945
acceptance probablity: 0.0005438972401216038
Chain length: 6
beta proposal: [0.5752814  0.02216814]
phi proposal: 0.5167416297254364
acceptance probablity: 6.700636639269737e-18
Chain length: 7
beta proposal: [0.81421988 0.12553216]
phi proposal: 0.8372135603345351
acceptance probab

curious about the following 
is there a ways to restrict the range of the sampling when we know that it is good? 

In [62]:
from concurrent.futures import ProcessPoolExecutor

def run_single_chain(seed, beta_true, number_of_iter, burn_in, sigma, num_points):
    # Set the random seed to ensure independent chains.
    np.random.seed(42)
    return MCMC(beta_true, number_of_iter, burn_in, sigma, num_points)

def run_multiple_chains(n_chains, beta_true, number_of_iter, burn_in, sigma, num_points):
    seeds = np.random.randint(0, 10000, size=n_chains)
    chains = []
    beta_mcmcs = []
    acceptance_histories = []
    acceptance_counts = []
    
    # Run chains in parallel using ProcessPoolExecutor
    with ProcessPoolExecutor() as executor:
        # Submit all the chains concurrently
        futures = [
            executor.submit(run_single_chain, seed, beta_true, number_of_iter, burn_in, sigma, num_points)
            for seed in seeds
        ]
        # Collect results as they complete
        for future in futures:
            chain, beta_mcmc, acceptance_prob_history, acceptance_count = future.result()
            chains.append(chain)
            beta_mcmcs.append(beta_mcmc)
            acceptance_histories.append(acceptance_prob_history)
            acceptance_counts.append(acceptance_count)
    
    return chains, beta_mcmcs, acceptance_histories, acceptance_counts

if __name__ == '__main__':
    n_chains = 4
    number_of_iter = 10000
    burn_in = 1000
    sigma = 0.1 
    num_points = 100
    beta_true = np.array([0.5, 1/6])
    chains, beta_mcmcs, acceptance_histories, acceptance_counts = run_multiple_chains(
        n_chains, beta_true, number_of_iter, burn_in, sigma, num_points
    )
print("True beta:", beta_true )
print("MCMC estimated beta:", beta_mcmcs)
print("Acceptance Probability History:", acceptance_histories)
print("Acceptance Count:", acceptance_counts)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.